In [54]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

# Load dataset
dataset = pd.read_csv("Dataset.csv")

# Convert 'Time' to datetime format
dataset['Time'] = pd.to_datetime(dataset['Time'], format='%d/%b/%Y:%H:%M:%S %z')

# Feature Engineering
# 1. Frekuensi Akses Per IP
dataset['Request_Per_Minute'] = dataset.groupby('IP')['Time'].transform(
    lambda x: x.diff().dt.total_seconds().fillna(60).rsub(60).clip(lower=0).gt(0).cumsum()
)

# 2. Pola Waktu Akses
dataset['Time_Difference'] = dataset.groupby('IP')['Time'].diff().dt.total_seconds().fillna(0)

# Deteksi Pola Serangan
# 1. SQL Injection
sql_pattern = re.compile(r'.*(SELECT|UNION|INSERT|DELETE|DROP).*', re.IGNORECASE)
dataset['SQL_Injection'] = dataset['URL'].apply(lambda x: bool(sql_pattern.search(x)))

# 2. XSS Detection
xss_pattern = re.compile(r'.*(<script>|%3Cscript%3E).*', re.IGNORECASE)
dataset['XSS'] = dataset['URL'].apply(lambda x: bool(xss_pattern.search(x)))

# 3. Directory Traversal
dir_traversal_pattern = re.compile(r'.*\.\./.*')
dataset['Directory_Traversal'] = dataset['URL'].apply(lambda x: bool(dir_traversal_pattern.search(x)))

# 4. DoS Detection
# Rule: Requests from the same IP within 0.5 seconds
dataset['DoS'] = dataset.groupby('IP')['Time_Difference'].transform(lambda x: (x <= 0.5))

# Pastikan kolom Status diubah menjadi string
dataset['Status'] = dataset['Status'].astype(str)

# Brute Force Detection
# Rule: More than X failed attempts from the same IP
failed_attempts = dataset['Status'].apply(lambda x: x.startswith('4'))
dataset['Brute_Force'] = dataset.groupby('IP')['Status'].transform(
    lambda x: x.str.startswith('4').astype(int).rolling(window=10, min_periods=1).sum().gt(5)
)


# Normalisasi Data
numerical_features = ['Request_Per_Minute', 'Time_Difference']
dataset[numerical_features] = (dataset[numerical_features] - dataset[numerical_features].mean()) / dataset[numerical_features].std()

# Save the processed dataset
dataset.to_csv("processed_dataset.csv", index=False)

print("Processing complete. Processed dataset saved as 'processed_dataset.csv'.")


Processing complete. Processed dataset saved as 'processed_dataset.csv'.


In [55]:
import pandas as pd

# Baca file CSV
file_path = 'processed_dataset.csv'  
data = pd.read_csv(file_path)

# Tampilkan beberapa baris pertama untuk memastikan data terbaca dengan benar
print(data.head())


              IP User                       Time Method URL  Status  Size  \
0  10.252.252.30    -  2023-11-10 11:02:00+07:00    GET   /     500  2527   
1  10.252.252.30    -  2023-11-10 11:02:01+07:00    GET   /     500  2527   
2  10.252.252.30    -  2023-11-10 11:06:47+07:00    GET   /     500  2527   
3  10.252.252.30    -  2023-11-10 11:06:48+07:00    GET   /     500  2527   
4  10.252.252.30    -  2023-11-10 11:11:42+07:00    GET   /     302     0   

  Referer                                              Agent   Hasil  \
0       -  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...  Normal   
1       -  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...  Normal   
2       -  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...  Normal   
3       -  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...  Normal   
4       -  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...  Normal   

   Request_Per_Minute  Time_Difference  SQL_Injection    XSS  \
0           -1.723496        -0.064768  

In [56]:
# Deteksi SQL Injection
import re

sql_pattern = re.compile(r".*(UNION|SELECT|INSERT|DELETE|UPDATE).*", re.IGNORECASE)
data['Detected_SQL_Injection'] = data['URL'].apply(lambda x: bool(sql_pattern.match(x)))

# Deteksi XSS
xss_pattern = re.compile(r".*(<script>|%3Cscript%3E).*", re.IGNORECASE)
data['Detected_XSS'] = data['URL'].apply(lambda x: bool(xss_pattern.match(x)))

# Deteksi Directory Traversal
dir_traversal_pattern = re.compile(r".*(\.\./|\.\.\\).*", re.IGNORECASE)
data['Detected_Directory_Traversal'] = data['URL'].apply(lambda x: bool(dir_traversal_pattern.match(x)))

# Gabungkan hasil ke dalam dataset
data['Any_Attack_Detected'] = data[['Detected_SQL_Injection', 'Detected_XSS', 'Detected_Directory_Traversal', 'DoS', 'Brute_Force']].any(axis=1)

# Tampilkan hasil deteksi
print(data[['IP', 'URL', 'Detected_SQL_Injection', 'Detected_XSS', 'Detected_Directory_Traversal', 'DoS', 'Brute_Force', 'Any_Attack_Detected']])


                  IP                                                URL  \
0      10.252.252.30                                                  /   
1      10.252.252.30                                                  /   
2      10.252.252.30                                                  /   
3      10.252.252.30                                                  /   
4      10.252.252.30                                                  /   
...              ...                                                ...   
99995  10.252.252.57                                                  /   
99996  10.252.252.57  /wp-cron.php?doing_wp_cron=1703488804.87900805...   
99997  10.252.252.57                                                  /   
99998  10.252.252.57                                                  /   
99999  10.252.252.57  /tiga-mahasiswa-ittelkom-surabaya-borong-pengh...   

       Detected_SQL_Injection  Detected_XSS  Detected_Directory_Traversal  \
0                     

In [57]:
from sklearn.model_selection import train_test_split

# Pisahkan fitur (X) dan label (y)
X = data[['Request_Per_Minute', 'Time_Difference']]  # Ganti dengan fitur yang relevan
y = data['Any_Attack_Detected']  # Target label

# Bagi data menjadi training (70%) dan testing (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Tampilkan ukuran dataset
print(f"Training set: {X_train.shape}, Testing set: {X_test.shape}")


Training set: (70000, 2), Testing set: (30000, 2)


In [58]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Latih model Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Prediksi dan evaluasi
y_pred_rf = rf.predict(X_test)
print("Random Forest:")
print(classification_report(y_test, y_pred_rf))


Random Forest:
              precision    recall  f1-score   support

       False       0.99      1.00      0.99     18021
        True       0.99      0.99      0.99     11979

    accuracy                           0.99     30000
   macro avg       0.99      0.99      0.99     30000
weighted avg       0.99      0.99      0.99     30000



In [59]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Latih model Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Prediksi dan evaluasi
y_pred_logreg = logreg.predict(X_test)
print("Logistic Regression:")
print(classification_report(y_test, y_pred_logreg))


Logistic Regression:
              precision    recall  f1-score   support

       False       0.69      0.80      0.74     18021
        True       0.60      0.45      0.51     11979

    accuracy                           0.66     30000
   macro avg       0.65      0.63      0.63     30000
weighted avg       0.65      0.66      0.65     30000



In [60]:
from sklearn.tree import DecisionTreeClassifier

# Latih model Decision Tree
tree = DecisionTreeClassifier(max_depth=5, random_state=42)
tree.fit(X_train, y_train)

# Prediksi dan evaluasi
y_pred_tree = tree.predict(X_test)
print("Decision Tree:")
print(classification_report(y_test, y_pred_tree))


Decision Tree:
              precision    recall  f1-score   support

       False       0.98      1.00      0.99     18021
        True       1.00      0.97      0.98     11979

    accuracy                           0.99     30000
   macro avg       0.99      0.98      0.99     30000
weighted avg       0.99      0.99      0.99     30000



In [61]:
from sklearn.svm import SVC

# Latih model SVM
svm = SVC(kernel='rbf', C=1, gamma=0.1)
svm.fit(X_train, y_train)

# Prediksi dan evaluasi
y_pred_svm = svm.predict(X_test)
print("Support Vector Machine (SVM):")
print(classification_report(y_test, y_pred_svm))


Support Vector Machine (SVM):
              precision    recall  f1-score   support

       False       0.70      0.71      0.70     18021
        True       0.55      0.54      0.55     11979

    accuracy                           0.64     30000
   macro avg       0.63      0.62      0.62     30000
weighted avg       0.64      0.64      0.64     30000



In [62]:
from xgboost import XGBClassifier

# Latih model XGBoost
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

# Prediksi dan evaluasi
y_pred_xgb = xgb.predict(X_test)
print("XGBoost:")
print(classification_report(y_test, y_pred_xgb))


XGBoost:
              precision    recall  f1-score   support

       False       0.99      0.99      0.99     18021
        True       0.99      0.98      0.98     11979

    accuracy                           0.99     30000
   macro avg       0.99      0.99      0.99     30000
weighted avg       0.99      0.99      0.99     30000



In [63]:
from sklearn.neural_network import MLPClassifier

# Latih model Neural Network
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
mlp.fit(X_train, y_train)

# Prediksi dan evaluasi
y_pred_mlp = mlp.predict(X_test)
print("Neural Network:")
print(classification_report(y_test, y_pred_mlp))


Neural Network:
              precision    recall  f1-score   support

       False       0.97      1.00      0.99     18021
        True       1.00      0.96      0.98     11979

    accuracy                           0.98     30000
   macro avg       0.99      0.98      0.98     30000
weighted avg       0.98      0.98      0.98     30000



In [69]:
from sklearn.neighbors import KNeighborsClassifier

# Latih model KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Prediksi dan evaluasi
y_pred_knn = knn.predict(X_test)
print("K-Nearest Neighbors (KNN):")
print(classification_report(y_test, y_pred_knn))


K-Nearest Neighbors (KNN):
              precision    recall  f1-score   support

       False       0.99      1.00      0.99     18021
        True       0.99      0.99      0.99     11979

    accuracy                           0.99     30000
   macro avg       0.99      0.99      0.99     30000
weighted avg       0.99      0.99      0.99     30000

